In [1]:
import sys, os
sys.path.append(os.path.realpath(".."))
os.environ["TF_XLA_FLAGS"]="--tf_xla_cpu_global_jit"

import util_funcs
from importlib import reload
import data_reader as read
import pandas as pd
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
import constants
import clinical_text_analysis as cta
import tsfresh
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, roc_curve
from os import path
import predictSeizureConvExp as psce
import keras_models.dataGen as dg
from addict import Dict
reload(psce)
from keras.utils import multi_gpu_model
import keras.optimizers
import time
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score, log_loss

Using TensorFlow backend.


In [2]:
config = Dict(psce.standardized_ensemble())
config.keys()

dict_keys(['use_random_ensemble', 'precached_pkl', 'precached_test_pkl', 'ensemble_sample_info_path', 'max_num_samples', 'use_standard_scaler', 'use_filtering'])

In [3]:
precached_pkl = "/n/scratch2/ms994/standardized_simple_ensemble_train_data_seizure.pkl"

In [4]:
trainDataGen = psce.get_data_generator("train", config.precached_pkl)

In [5]:
testDataGen = psce.get_data_generator("dev_test", config.precached_test_pkl, is_test=True)

In [6]:
trainDataGen, validDataGen = trainDataGen

In [30]:
len(testDataGen)

1093

In [7]:
testX = np.stack([datum[0] for datum in testDataGen.dataset])
testY = np.stack([datum[1] for datum in testDataGen.dataset])

In [39]:
testX.shape, testY.shape

((34998, 500, 21), (34998,))

In [8]:
seizureX = np.vstack([testX[testY == 1], testX[0:2500]])
seizureX.shape

(4984, 500, 21)

In [87]:
seizureY = np.hstack([testY[testY == 1], testY[0:2500]])
seizureY.shape, seizureY.mean()

((5018,), 0.5245117576723795)

In [ ]:
seizy

In [9]:
import keras_models.vanPutten as vp

In [21]:
reload(vp)
model = vp.inception_like((21,500,1), num_layers=4, num_filters=30, dropout=0.75)

In [22]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 21, 500, 1)   0                                            
__________________________________________________________________________________________________
conv2d_21 (Conv2D)              (None, 20, 499, 30)  150         input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_25 (Conv2D)              (None, 19, 498, 30)  300         input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_29 (Conv2D)              (None, 18, 497, 30)  510         input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_33 

In [23]:
model = multi_gpu_model(model, 8)

In [24]:
adam = keras.optimizers.Adam(lr=0.002)
model.compile(adam, loss="categorical_crossentropy")

In [ ]:
validData = validDataGen.dataset

In [ ]:
validDataX = np.stack([datum[0] for datum in validData])
validDataY = np.stack([datum[1] for datum in validData])

In [27]:
len(validDataGen.dataset)

17965

In [28]:
validDataGen.batch_size = 256*8

In [ ]:
from keras_models import train as util_run
reload(util_run)
history = util_run.model_run(model=model,
                             patience=10,
                             num_steps_each_eval=32,
                             class_weights=[1,40000],
                             model_name="yolo1.h5",
                             epochs=100,
                             trainDataGen=trainDataGen,
                             validDataGen=validDataGen, 
                             verbosity=True,
                             update_amount=0.97
                            )

Progress: 0/3267
Epoch: 0, Step: 0/32, Loss: 1.0073810815811157
predicted all 0
Updating Class Weight to: [9.00000000e-01 4.44444444e+04]
f1 increased from -1 to 0.0


/home/ms994/miniconda3/envs/keras-redo/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Progress: 1/3267
Epoch: 0, Step: 1/32, Loss: 1.0073810815811157
Progress: 2/3267
Epoch: 0, Step: 2/32, Loss: 1.1920930376163597e-07
Progress: 3/3267
Epoch: 0, Step: 3/32, Loss: 1.0073810815811157
Progress: 4/3267
Epoch: 0, Step: 4/32, Loss: 0.5036906003952026
Progress: 5/3267
Epoch: 0, Step: 5/32, Loss: 1.1920930376163597e-07
Progress: 6/3267
Epoch: 0, Step: 6/32, Loss: 0.5036906003952026
Progress: 7/3267
Epoch: 0, Step: 7/32, Loss: 0.5036906003952026
Progress: 8/3267
Epoch: 0, Step: 8/32, Loss: 0.5036906003952026
Progress: 9/3267
Epoch: 0, Step: 9/32, Loss: 1.0073810815811157
Progress: 10/3267
Epoch: 0, Step: 10/32, Loss: 1.5110714435577393
Progress: 11/3267
Epoch: 0, Step: 11/32, Loss: 1.1920930376163597e-07
Progress: 12/3267
Epoch: 0, Step: 12/32, Loss: 1.5110714435577393
Progress: 13/3267
Epoch: 0, Step: 13/32, Loss: 1.0073810815811157
Progress: 14/3267
Epoch: 0, Step: 14/32, Loss: 0.5036906003952026
Progress: 15/3267
Epoch: 0, Step: 15/32, Loss: 1.1920930376163597e-07
Progress: 16

/home/ms994/miniconda3/envs/keras-redo/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Progress: 33/3267
Epoch: 0, Step: 33/32, Loss: 0.5036906003952026
Progress: 34/3267
Epoch: 0, Step: 34/32, Loss: 2.0147619247436523
Progress: 35/3267
Epoch: 0, Step: 35/32, Loss: 0.5036906003952026
Progress: 36/3267
Epoch: 0, Step: 36/32, Loss: 1.0073810815811157
Progress: 37/3267
Epoch: 0, Step: 37/32, Loss: 0.5036906003952026
Progress: 38/3267
Epoch: 0, Step: 38/32, Loss: 1.0073810815811157
Progress: 39/3267
Epoch: 0, Step: 39/32, Loss: 1.1920930376163597e-07
Progress: 40/3267
Epoch: 0, Step: 40/32, Loss: 1.0073810815811157
Progress: 41/3267
Epoch: 0, Step: 41/32, Loss: 1.0073810815811157
Progress: 42/3267
Epoch: 0, Step: 42/32, Loss: 1.0073810815811157
Progress: 43/3267
Epoch: 0, Step: 43/32, Loss: 1.0073810815811157
Progress: 44/3267
Epoch: 0, Step: 44/32, Loss: 1.5110715627670288
Progress: 45/3267
Epoch: 0, Step: 45/32, Loss: 1.5110714435577393
Progress: 46/3267
Epoch: 0, Step: 46/32, Loss: 1.1920930376163597e-07
Progress: 47/3267
Epoch: 0, Step: 47/32, Loss: 0.5036906003952026
Pr

In [56]:
len(history.history.loss)

2

In [102]:
trainDataGen.batch_size = 1280

In [103]:
start = time.time()
lossHist = []
callback_check = 10
epochs=1
# seizureX = seizureX.reshape((*seizureX.shape, 1)).transpose((0,2,1,3))
num_steps = len(trainDataGen)
# num_steps = 10
validX = []
validY = []

In [106]:
callback_check = 5
start = time.time()
class_weight = np.array([1,20])
update_amount = 0.9
for i in range((epochs)):
    for j in range(num_steps):
        loss = model.train_on_batch(trainDataGen[j][0], trainDataGen[j][1], class_weight=class_weight)
        lossHist.append(loss)
        print("Epoch: {}, Step: {}/{}, Loss: {}".format(i,j, num_steps, loss))
        if j % callback_check == 0:
            valid_index = np.random.choice(list(range(len(seizureX))), 1000)
            validX = seizureX[valid_index]
            validY = seizureY[valid_index]
            y_pred = model.predict(validX)
            y_pred = y_pred.argmax(1)
            if y_pred.mean() == int(y_pred.mean()):
                problem_class = int(y_pred.mean())
                class_weight = class_weight / update_amount
                class_weight[problem_class] = class_weight[problem_class] * update_amount ** 2
                print("predicted all {}".format(problem_class))
                print("Updating Class Weight to: {}".format(class_weight))

print(time.time()-start)

Epoch: 0, Step: 0/95, Loss: 0.2845654785633087
predicted all 0
Updating Class Weight to: [ 0.9        22.22222222]
Epoch: 0, Step: 1/95, Loss: 0.26884907484054565
Epoch: 0, Step: 2/95, Loss: 0.3087906539440155
Epoch: 0, Step: 3/95, Loss: 0.30356794595718384
Epoch: 0, Step: 4/95, Loss: 0.24307191371917725
Epoch: 0, Step: 5/95, Loss: 0.29212602972984314
predicted all 0
Updating Class Weight to: [ 0.81       24.69135802]
Epoch: 0, Step: 6/95, Loss: 0.2752738893032074
Epoch: 0, Step: 7/95, Loss: 0.255071759223938
Epoch: 0, Step: 8/95, Loss: 0.25660499930381775
Epoch: 0, Step: 9/95, Loss: 0.2733497619628906
Epoch: 0, Step: 10/95, Loss: 0.19150973856449127
predicted all 0
Updating Class Weight to: [ 0.729      27.43484225]
Epoch: 0, Step: 11/95, Loss: 0.2439744919538498
Epoch: 0, Step: 12/95, Loss: 0.247213214635849
Epoch: 0, Step: 13/95, Loss: 0.2706335186958313
Epoch: 0, Step: 14/95, Loss: 0.3031953275203705
Epoch: 0, Step: 15/95, Loss: 0.28288760781288147
predicted all 0
Updating Class We

In [107]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [84]:
class_weight = class_weight / 0.75

In [78]:
1.0 == int(1)

True

In [109]:
y_pred = model.predict(seizureX)

In [114]:
confusion_matrix(seizureY, y_pred.argmax(1),)

array([[2386,    0],
       [2629,    3]])

AttributeError: 'numpy.ndarray' object has no attribute 'get_shape'